## 月MACD在零线上刚金叉的

In [3]:
import akshare as ak
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import os

current_date = datetime.now() + timedelta(days=1)
one_week_ago = current_date - timedelta(days=7)  # 计算一周之前的日期

In [4]:
# 生成股票代码 名称 词典
def generate_stock_dict(filename: str):
    stock_dict = {}

    with open(filename, "r", encoding="utf-8") as file:
        # 读取文件内容
        content = file.read()

        # 按照股票名和股票代码的格式分割
        stock_info = content.split(")")  # 按照')'分割

        for info in stock_info:
            if "(" in info:  # 如果包含'('，说明这是一个有效的股票信息
                code_name = info.split("(")  # 通过'('分割代码和名称
                if len(code_name) == 2:
                    code = code_name[1]
                    name = code_name[0].strip()
                    stock_dict[code] = name


    return stock_dict
# 带有重试机制的 yfinance 数据下载函数。
def download_with_retry(stock_code: str, max_retries=2, retry_delay=3):
    end_date = current_date
    for attempt in range(max_retries):
        try:
            df = ak.stock_zh_a_cdr_daily(
                symbol=stock_code,
                start_date="2010-01-01",
                end_date=end_date.strftime("%Y-%m-%d"),
            )
            if not df.empty:  # 检查数据是否下载成功
                return df
            else:
                raise ValueError("Empty DataFrame returned.")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {stock_code}: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)  # 等待后重试
                end_date = end_date - timedelta(1)
            else:
                print(f"Max retries reached for {stock_code}. Skipping...")
    return pd.DataFrame()  # 如果失败，返回空 DataFrame


# 输出最终excel文件
def output_excel(all_cross: list, output_file: str):
    filename = "stock_name.txt"  # 替换新文件路径
    stock_dict = generate_stock_dict(filename)
    num = len(all_cross)
    data = []

    # 遍历所有股票金叉数据
    for stock_code in all_cross:
        cur_stock_code = str(stock_code[-6:])  # 去掉市场后缀，保留6位股票代码
        stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
        data.append({"股票代码": cur_stock_code, "股票名称": stock_name})

    df = pd.DataFrame(data)
    summary_row = pd.DataFrame(
        {"股票代码": [f"总计金叉股票个数：{num}"], "股票名称": [""]}
    )  # 添加总计行到 DataFrame 的顶部

    df = pd.concat([summary_row, df], ignore_index=True)

    # output_file = ".\golden_cross_summary1.xlsx"
    df.to_excel(output_file, index=False, sheet_name="金叉统计")
    print(f"Excel 文件已生成: {output_file}")


def clean_folder(folder_path: str):
    """删除指定文件夹中的所有文件"""
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")


def mk_pic(output_file: str):
    # 读取 Excel
    file_path = output_file
    df = pd.read_excel(file_path, usecols=[0])  # 只读取第一列
    directory = os.path.dirname(output_file)  # 提取目录部分

    # 解决中文乱码
    plt.rcParams["font.sans-serif"] = ["SimHei"]
    plt.rcParams["axes.unicode_minus"] = False

    # 设置每张图最多显示的行数
    rows_per_image = 50

    # 按每 50 行划分数据并生成图片
    for i in range(0, len(df), rows_per_image):
        subset = df.iloc[i : i + rows_per_image]  # 取出 50 行数据
        fig, ax = plt.subplots(figsize=(5, len(subset) * 0.4))  # 调整图片大小
        ax.axis("tight")
        ax.axis("off")
        ax.table(
            cellText=subset.values,
            colLabels=subset.columns,
            cellLoc="center",
            loc="center",
        )

        # 生成文件名，如 output_0.png, output_1.png
        output_path = f"{directory}\output_{i // rows_per_image}.png"
        plt.savefig(output_path, bbox_inches="tight", dpi=300)

        # 关闭图像，释放内存
        plt.close(fig)

    print("所有图片已生成！")

In [5]:
# 传统方式计算EMA
def calculate_ema(data: DataFrame, span):
    alpha = 2 / (span + 1)
    # 使用第一个值作为初始EMA
    ema = data.copy()
    ema.iloc[0] = data.iloc[0]

    # 逐个计算后续EMA
    for i in range(1, len(data)):
        ema.iloc[i] = alpha * data.iloc[i] + (1 - alpha) * ema.iloc[i - 1]

    return ema


# 传统方法计算MACD
def calculate_macd(df: DataFrame, short=12, long=26, signal=9):
    df["EMA_short"] = calculate_ema(df["close"], short)
    df["EMA_long"] = calculate_ema(df["close"], long)
    df["DIF"] = df["EMA_short"] - df["EMA_long"]
    df["DEA"] = calculate_ema(df["DIF"], signal)
    df["MACD_hist"] = 2 * (df["DIF"] - df["DEA"])

    # 计算金叉
    df["MACD_Golden"] = (df["DIF"].shift(1) <= df["DEA"].shift(1)) & (
        df["DIF"] > df["DEA"]
    )
    df["MACD_Death"] = (df["DIF"].shift(1) >= df["DEA"].shift(1)) & (
        df["DIF"] < df["DEA"]
    )
    return df

In [6]:
# 将日线数据转换为月线数据
def get_monthly_kline(df: DataFrame):
    # 确保日期列是datetime类型
    if not pd.api.types.is_datetime64_any_dtype(df["date"]):
        df["date"] = pd.to_datetime(df["date"])

    # 添加年月列
    df["year_month"] = df["date"].dt.to_period("M")

    # 生成月线数据
    monthly_df = (
        df.groupby("year_month")
        .agg(
            {
                "open": "first",  # 月开盘价取第一个交易日
                "high": "max",  # 月最高价取最高点
                "low": "min",  # 月最低价取最低点
                "close": "last",  # 月收盘价取最后一个交易日
                "volume": "sum",  # 月成交量取总和
            }
        )
        .reset_index()
    )

    # 将year_month转换为datetime以便后续处理
    monthly_df["date"] = monthly_df["year_month"].dt.to_timestamp()

    return monthly_df

# 检查DIFF > 0
def check_diff_pos(df: DataFrame):
    if df["DIF"].iloc[-2] < 0 and df["DIF"].iloc[-1] > 0:
        return True
    return False


# 检查最近月MACD是否为金叉 以及 DIFF > 0
def check_monthly_macd_golden_cross(stock_code: str, df: DataFrame):
    # 转换为月线数据
    monthly_df = get_monthly_kline(df)

    # 计算月MACD
    monthly_df = calculate_macd(monthly_df)

    # 检查最近月MACD是否为金叉 以及 DIFF > 0
    if get_golden_periods(monthly_df) and check_diff_pos(monthly_df):
        return stock_code

    return None


# 查找最近金叉时段
def get_golden_periods(df: DataFrame):
    periods = []
    start_date = None  # 用于记录金叉的开始日期
    end_date = df["year_month"].iloc[-1]

    for i in range(len(df)):
        if df["MACD_Golden"].iloc[i]:  # 如果是金叉
            start_date = df["year_month"].iloc[i]  # 开始记录金叉日期
        elif df["MACD_Death"].iloc[i]:  # 如果是死叉
            start_date = None  # 重置金叉开始日期

    # 如果循环结束时还有未匹配的金叉（没有死叉）
    if start_date is not None:
        periods.append([start_date, end_date])  # 配对到最近日期

    return periods  # 返回金叉日期段


# 主函数,获取股票的MACD金叉情况
def get_macd(stock_code: str):
    try:
        # 下载股票日线数据
        data = download_with_retry(stock_code)

        # 检查月MACD金叉
        result = check_monthly_macd_golden_cross(stock_code, data)
        return result
    except Exception as e:
        print(f"处理股票 {stock_code} 时出错: {e}")
        return None


def process_stock(stock_code:str):
    time.sleep(random.uniform(0.1, 0.2))  # 随机延时
    try:
        stock_code = get_macd(stock_code)
        return stock_code
    except Exception as e:
        print(f"Error processing {stock_code}: {e}")
        return stock_code, []


# 替换原来的循环处理部分
def process_stocks(stock_list: list):
    all_golden_cross = []

    # 创建线程池，max_workers 可以根据需要调整
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(process_stock, stock): stock for stock in stock_list}
        for future in as_completed(futures):
            result = future.result()  # 这里获取 process_stock 的返回值
            if result:  # 确保返回值不是 None
                all_golden_cross.append(result)

    return all_golden_cross  # 传统方式计算EMA

In [7]:
file_name = "stock_code.txt"
with open(file_name, "r") as file:
    stock_list = [line.strip() for line in file if line.strip()]

# 限制股票数量
stock_list = stock_list[:250]

all_golden_cross = process_stocks(stock_list)

In [8]:
filename = "stock_name.txt"  # 替换新文件路径
stock_dict = generate_stock_dict(filename)
print(f"总计股票数量：{len(all_golden_cross)}")
for stock_code in all_golden_cross:
    cur_stock_code = str(stock_code[-6:])  # 取出六位股票代码
    stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
    print(
        f"股票代码: {cur_stock_code}, 股票名称: {stock_name} "
    )

总计股票数量：12
股票代码: 600031, 股票名称: 三一重工 
股票代码: 600064, 股票名称: 南京高科 
股票代码: 600080, 股票名称: 金花股份 
股票代码: 600143, 股票名称: 金发科技 
股票代码: 600157, 股票名称: 永泰能源 
股票代码: 600169, 股票名称: 太原重工 
股票代码: 600173, 股票名称: 卧龙地产 
股票代码: 600215, 股票名称: 派斯林 
股票代码: 600232, 股票名称: 金鹰股份 
股票代码: 600289, 股票名称: *ST信通 
股票代码: 600300, 股票名称: 维维股份 
股票代码: 600293, 股票名称: 三峡新材 


In [9]:
try:
    folder = Path("month_macd")
    folder.mkdir()
except Exception as e:
    pass

clean_folder(".\month_macd")

output_file = ".\month_macd\month_cross_summary.xlsx"
output_excel(all_golden_cross, output_file)  # 生成excel
mk_pic(output_file)  # 生成所有图片

Excel 文件已生成: .\month_macd\month_cross_summary.xlsx
所有图片已生成！


In [22]:
stock_code = "sh600031"
data = download_with_retry(stock_code)

In [20]:
# 创建月度数据
monthly_df = data.copy()
monthly_df = get_monthly_kline(monthly_df)

In [21]:
monthly_df.tail(12)

,year_month,open,high,low,close,volume,date
171,2024-04,14.65,16.49,14.61,16.31,1.761313e+09,2024-04-01
172,2024-05,16.54,17.57,16.09,16.18,1.257769e+09,2024-05-01
173,2024-06,16.18,16.62,15.42,16.50,1.063333e+09,2024-06-01
174,2024-07,16.46,16.60,14.88,16.15,9.499019e+08,2024-07-01
175,2024-08,16.09,16.25,14.85,16.12,6.792911e+08,2024-08-01
176,2024-09,16.16,19.20,15.07,18.88,1.158808e+09,2024-09-01
177,2024-10,20.76,20.76,17.26,18.25,1.807381e+09,2024-10-01
178,2024-11,18.21,19.12,16.97,17.48,1.493739e+09,2024-11-01
179,2024-12,17.57,17.58,16.08,16.48,1.217889e+09,2024-12-01
180,2025-01,16.48,16.48,15.23,16.10,9.278932e+08,2025-01-01


In [27]:
# 计算月线数据（仅提取每月最后一个交易日）
def get_monthly_kline(df: pd.DataFrame):
    """提取每月最后一个交易日的数据"""
    df["date"] = pd.to_datetime(df["date"])  # 确保日期格式正确
    df["year_month"] = df["date"].dt.to_period("M")  # 按年月分组

    # 选取每月最后一个交易日的数据
    last_day_df = df.loc[df.groupby("year_month")["date"].idxmax()].reset_index(
        drop=True
    )

    # 选择所需列
    monthly_df = last_day_df[["date", "open", "high", "low", "close", "volume"]].copy()

    return monthly_df


# 示例：假设 df 是日线数据
df = data.copy()
monthly_df = get_monthly_kline(df)  # 提取每月最后一个交易日的收盘价
monthly_df = calculate_macd(monthly_df)  # 计算月线 MACD

In [28]:
monthly_df.tail(7)

,date,open,high,low,close,volume,EMA_short,EMA_long,DIF,DEA,MACD_hist,MACD_Golden,MACD_Death
176,2024-09-30,18.42,19.20,18.02,18.88,248847666.0,16.165756,16.608144,-0.442388,-1.016523,1.148270,False,False
177,2024-10-31,17.66,18.56,17.66,18.25,152303103.0,16.486409,16.729763,-0.243354,-0.861889,1.237071,False,False
178,2024-11-29,17.33,17.65,17.25,17.48,59049392.0,16.639269,16.785336,-0.146067,-0.718725,1.145316,False,False
179,2024-12-31,16.66,16.75,16.48,16.48,51515564.0,16.614766,16.762718,-0.147952,-0.604570,0.913236,False,False
180,2025-01-27,16.29,16.34,16.10,16.10,30223469.0,16.535571,16.713628,-0.178057,-0.519267,0.682421,False,False
181,2025-02-28,17.93,18.50,17.91,18.20,137167392.0,16.791637,16.823730,-0.032092,-0.421832,0.779480,False,False
182,2025-03-17,20.48,20.59,20.19,20.43,109203646.0,17.351385,17.090861,0.260525,-0.285361,1.091771,False,False
